### Module Import and Auto-Reload

In [1]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


In [31]:

from src.Agents.call_center_agent.data.client_data_fetcher import  get_client_data

client_data = get_client_data('23001', force_reload=True)

2025-05-29 16:39:32,273 - INFO - Data loaded for user 23001 - 0 failures


In [30]:
# client_data

In [32]:
# Determine script type from aging
account_aging = client_data.get("account_aging", {})
account_aging

{'x0': '0.0000',
 'x30': '179.0000',
 'x60': '0.0000',
 'x90': '131.0000',
 'x120': '0.0000',
 'xbalance': '310.0000'}

In [17]:
payment_history = client_data.get("payment_history", [])
# payment_history.get("arrangement_state")
payment_history

[{'arrangement_id': 381875233,
  'user_id': 83906,
  'note': 'Promise to Pay: ZAR550.00 2025-05-09. Comment: n/a',
  'create_ts': '2025-05-09T11:16:59.718858+02:00',
  'followup_ts': '2025-05-10T00:00:00+02:00',
  'created_by': 'ai_agent',
  'arrangement_state': 'Failed',
  'arrangement_state_type': 'Closed',
  'amount': '550.00',
  'pay_date': '2025-05-09',
  'arrangement_pay_type': 'Manual Debit Order',
  'mandate_fee': None,
  'cheque_number': None,
  'online_payment_reference_id_original': None},
 {'arrangement_id': 381875231,
  'user_id': 83906,
  'note': 'Promise to Pay: ZAR550.00 2025-05-09',
  'create_ts': '2025-05-09T11:06:11.825756+02:00',
  'followup_ts': '2025-05-10T00:00:00+02:00',
  'created_by': 'AI-System',
  'arrangement_state': 'Failed',
  'arrangement_state_type': 'Closed',
  'amount': '550.00',
  'pay_date': '2025-05-09',
  'arrangement_pay_type': 'Manual Debit Order',
  'mandate_fee': None,
  'cheque_number': None,
  'online_payment_reference_id_original': None},
 

In [22]:
from src.Agents.call_center_agent.call_scripts import ScriptManager
from src.Agents.call_center_agent.data.client_data_fetcher import  get_client_data

client_data = get_client_data('23001', force_reload=True)
account_aging = client_data.get("account_aging", {})
print(account_aging)

script_type = ScriptManager.determine_script_type_from_aging(account_aging, client_data)
aging_context = ScriptManager.get_aging_context(script_type)
script_type

2025-05-29 16:13:56,538 - INFO - Data loaded for user 23001 - 0 failures
2025-05-29 16:13:56,541 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0


{'x0': '0.0000', 'x30': '179.0000', 'x60': '0.0000', 'x90': '131.0000', 'x120': '0.0000', 'xbalance': '310.0000'}


<ScriptType.RATIO_2_3_INFLOW: 'ratio_2_3_inflow'>

In [29]:
from src.Agents.call_center_agent.step00_introduction import get_introduction_prompt
from src.Agents.call_center_agent.step01_name_verification import get_name_verification_prompt
from src.Agents.call_center_agent.step02_details_verification import get_details_verification_prompt
from src.Agents.call_center_agent.step03_reason_for_call import get_reason_for_call_prompt
from src.Agents.call_center_agent.step04_negotiation import get_negotiation_prompt
from src.Agents.call_center_agent.step05_promise_to_pay import get_promise_to_pay_prompt
from src.Agents.call_center_agent.step06_debicheck_setup import get_debicheck_setup_prompt
from src.Agents.call_center_agent.step07_payment_portal import get_payment_portal_prompt
from src.Agents.call_center_agent.step08_subscription_reminder import get_subscription_reminder_prompt
from src.Agents.call_center_agent.step09_client_details_update import get_client_details_update_prompt
from src.Agents.call_center_agent.step10_referrals import get_referrals_prompt
from src.Agents.call_center_agent.step11_further_assistance import get_further_assistance_prompt
from src.Agents.call_center_agent.step12_query_resolution import get_query_resolution_prompt
from src.Agents.call_center_agent.step13_escalation import get_escalation_prompt
from src.Agents.call_center_agent.step14_cancellation import get_cancellation_prompt
from src.Agents.call_center_agent.step15_closing import get_closing_prompt
from app_config import CONFIG # Assuming CONFIG is used for some default values or agent_name

# client_data = {
#     'profile': {
#         'client_info': {
#             'title': 'Mr',
#             'first_name': 'Sampie',
#             'client_full_name': 'Sampie BLOEM'
#         }
#     },
#     'account_overview': {
#         'xbalance': '310.00',
#         'subscription': '179.00',
#         'cancellation_fee': '1969.00'
#     },
#     'account_aging': {
#         'xbalance': '310.00'
#     },
#     'payment_history': [
#         {'amount_paid': '100.00', 'payment_date': '2025-05-20', 'agreed_amount': '200.00', 'shortfall': '100.00'}
#     ]
# }

# state = {
#     "agent_name": "Sarah",
#     "field_to_verify": "ID number",
#     "amount_with_fee": 320.00,
#     "outcome_summary": "We've resolved your payment for today."
# }

agent_name = "AI Agent" # Or retrieve from CONFIG if available, e.g., CONFIG.get("agent_name", "AI Agent")
state = {} # Initialize an empty state or with relevant default values

# --- List of prompt functions ---
# You can create a list of tuples, where each tuple is (step_name, prompt_function)
prompt_functions = [
    ("Introduction", get_introduction_prompt),
    ("Name Verification", get_name_verification_prompt),
    ("Details Verification", get_details_verification_prompt),
    ("Reason for Call", get_reason_for_call_prompt),
    ("Negotiation", get_negotiation_prompt),
    ("Promise to Pay", get_promise_to_pay_prompt),
    ("Debicheck Setup", get_debicheck_setup_prompt),
    ("Payment Portal", get_payment_portal_prompt),
    ("Subscription Reminder", get_subscription_reminder_prompt),
    ("Client Details Update", get_client_details_update_prompt),
    ("Referrals", get_referrals_prompt),
    ("Further Assistance", get_further_assistance_prompt),
    ("Query Resolution", get_query_resolution_prompt),
    ("Escalation", get_escalation_prompt),
    ("Cancellation", get_cancellation_prompt),
    ("Closing", get_closing_prompt),
]

# --- Loop to export (print) each prompt ---
print("--- Exporting Call Center Prompts ---")
print("="*40)

for step_name, prompt_func in prompt_functions:
    try:
        # Call the prompt function with your data
        if step_name == "Introduction":
            prompt_content = prompt_func(client_data, agent_name, state)
        else:
            prompt_content = prompt_func(client_data, state)

        print(f"\n## {step_name} Prompt ##")
        print("-" * (len(step_name) + 10)) # Decorative line based on title length
        print(prompt_content)
        print("\n" + "="*40 + "\n") # Separator

    except Exception as e:
        print(f"\n## Error exporting {step_name} Prompt ##")
        print(f"An error occurred: {e}")
        print("\n" + "="*40 + "\n")

print("--- Prompt Export Complete ---")

2025-05-29 16:36:53,972 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,973 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,973 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,973 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,974 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,974 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,974 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,974 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,975 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,975 - WARNING - Missing script parameter: 'subscription_date'. Using empty string as default.
2025-05-29 16:36:53,975 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:53,975 - INFO - Selected RATIO_2_3_INFLOW: x60=0.0, x90=131.0
2025-05-29 16:36:

--- Exporting Call Center Prompts ---

## Introduction Prompt ##
----------------------
<role>
You are AI Agent, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<task>
Deliver professional greeting and request specific client. MAXIMUM 15 words.
</task>

<response>
"Good day, you are speaking to AI Agent from Cartrack Accounts Department. May I speak to Mr Sampie BLOEM, please?"
</response>

<style>
- Professional and confident
- Clear company identification
- Direct request for specific person
- MAXIMUM 15 words
</style>

<script_context>
Script Type: ratio_2_3_inflow
Aging Category: 2-3 Months Overdue
Urgency Level: High
Recommended Tone: Serious and urgent
</script_context>

<script_content>
Good day, you are speaking to the Agent from Cartrack Accounts Department. May I speak to Mr Sampie BLOEM, please?
</script_content>

<aging_specific_approach>
Firm but solution-focused. Emphasize significant consequences and recovery fees.
</aging_specific_a

### Test

In [5]:
# ./example_usage_short.py
"""
Compact test to see full prompts for each call center step.
"""

from src.Agents.call_center_agent.call_scripts import ScriptType, CallStep
from src.Agents.call_center_agent.data_parameter_builder import (
    get_client_data,
    prepare_parameters, 
    ConversationState,
    VerificationStatus
)
from src.Agents.call_center_agent.prompts import get_step_prompt

def test_all_step_prompts():
    """Test all step prompts with full content display."""
    
    # Setup
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"
    
    print("🔄 Loading client data...")
    client_data = get_client_data(user_id)
    client_name = client_data.get('profile', {}).get('client_info', {}).get('client_full_name', 'Client')
    print(f"✅ Loaded data for: {client_name}")
    
    # Initialize state as fully verified
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    state.add_matched_field("id_number")
    state.add_objection("no_money")
    state.update_emotional_state("cooperative")
    
    # All steps to test
    steps = [
        "introduction",
        "name_verification", 
        "details_verification",
        "reason_for_call",
        "negotiation",
        "promise_to_pay",
        "debicheck_setup",
        "payment_portal",
        "subscription_reminder",
        "client_details_update",
        "referrals",
        "further_assistance",
        "query_resolution",
        "escalation",
        "cancellation",
        "closing"
    ]
    
    print(f"\n🧪 Testing {len(steps)} call center steps:")
    print("=" * 80)
    
    for i, step in enumerate(steps, 1):
        print(f"\n📋 STEP {i:2d}: {step.upper()}")
        print("-" * 50)
        
        try:
            # Build parameters
            parameters = prepare_parameters(
                client_data=client_data,
                current_step=step,
                state=state.to_dict(),
                script_type=script_type,
                agent_name=agent_name
            )
            
            # Generate full prompt
            full_prompt = get_step_prompt(step, parameters)
            
            # Display prompt (truncate if too long)
            if len(full_prompt) > 800:
                print(f"🎯 PROMPT: {full_prompt[:400]}...")
                print(f"...{full_prompt[-400:]}")
            else:
                print(f"🎯 PROMPT: {full_prompt}")
            
            # Show key parameters
            key_params = {
                "client_name": parameters.get("client_name", "N/A"),
                "outstanding_amount": parameters.get("outstanding_amount", "N/A"),
                "urgency_level": parameters.get("urgency_level", "N/A"),
                "key_motivators": parameters.get("key_motivators", "N/A")
            }
            print(f"📊 KEY PARAMS: {key_params}")
            
        except Exception as e:
            print(f"❌ ERROR: {e}")
        
        print("-" * 50)
    
    print(f"\n✅ Tested all {len(steps)} steps successfully!")

def test_script_variations_short():
    """Test different script types with key differences highlighted."""
    
    print("\n🎭 SCRIPT VARIATIONS TEST")
    print("=" * 60)
    
    user_id = "83906"
    agent_name = "Sarah"
    
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    script_types = [
        ("ratio_1_inflow", "First missed payment"),
        ("ratio_2_3_inflow", "2-3 months overdue"), 
        ("pre_legal_120_plus", "Pre-legal 120+ days"),
        ("pre_legal_150_plus", "Legal attorney involved")
    ]
    
    for script_type, description in script_types:
        print(f"\n📝 {description.upper()}")
        print("-" * 30)
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step="reason_for_call",
            state=state.to_dict(),
            script_type=script_type,
            agent_name=agent_name
        )
        
        # Show script content
        script_content = parameters.get("script_content", "No script content")
        print(f"💬 SCRIPT: {script_content}")
        
        # Show tactical differences
        urgency = parameters.get("urgency_level", "None")
        motivators = parameters.get("key_motivators", "None")
        
        print(f"⚡ URGENCY: {urgency}")
        print(f"🎯 MOTIVATORS: {motivators}")

def test_conversation_intelligence():
    """Test conversation intelligence features."""
    
    print("\n🧠 CONVERSATION INTELLIGENCE TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        detect_emotional_state,
        detect_objections,
        analyze_payment_conversation
    )
    
    # Simulate conversation messages
    test_messages = [
        {"role": "ai", "content": "Can we debit R500 today?"},
        {"role": "human", "content": "I don't have money right now, I'm really stressed about this"},
        {"role": "ai", "content": "I understand. What amount could you manage?"},
        {"role": "human", "content": "Maybe R200 next week when I get paid"}
    ]
    
    print("💭 Test conversation:")
    for msg in test_messages:
        speaker = "AGENT" if msg["role"] == "ai" else "CLIENT"
        print(f"   {speaker}: {msg['content']}")
    
    # Analyze conversation
    emotional_state = detect_emotional_state(test_messages)
    objections = detect_objections(test_messages)
    payment_analysis = analyze_payment_conversation(test_messages, 500.0)
    
    print(f"\n🎭 EMOTIONAL STATE: {emotional_state}")
    print(f"🚫 OBJECTIONS: {objections}")
    print(f"💰 PAYMENT ANALYSIS: {payment_analysis}")

def test_outstanding_calculation():
    """Test outstanding amount calculation."""
    
    print("\n💰 OUTSTANDING AMOUNT CALCULATION TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        calculate_outstanding_amount,
        format_outstanding_amount
    )
    
    # Test data
    test_aging_data = {
        "x0": "100.00",      # Current (not overdue)
        "x30": "200.00",     # 1-30 days overdue
        "x60": "150.00",     # 31-60 days overdue
        "x90": "100.00",     # 61-90 days overdue
        "x120": "50.00",     # 91+ days overdue
        "xbalance": "600.00" # Total balance
    }
    
    outstanding_float = calculate_outstanding_amount(test_aging_data)
    formatted_amount = format_outstanding_amount({"account_aging": test_aging_data})
    
    print(f"📊 AGING BREAKDOWN:")
    print(f"   Current (x0): R {test_aging_data['x0']}")
    print(f"   30 days: R {test_aging_data['x30']}")
    print(f"   60 days: R {test_aging_data['x60']}")
    print(f"   90 days: R {test_aging_data['x90']}")
    print(f"   120+ days: R {test_aging_data['x120']}")
    print(f"   TOTAL: R {test_aging_data['xbalance']}")
    
    print(f"\n💯 CALCULATION:")
    print(f"   Outstanding = Total - Current")
    print(f"   Outstanding = R {test_aging_data['xbalance']} - R {test_aging_data['x0']}")
    print(f"   Outstanding = R {outstanding_float:.2f}")
    print(f"   Formatted: {formatted_amount}")

def quick_test():
    """Quick test of key functionality."""
    
    print("🚀 QUICK CALL CENTER TEST")
    print("=" * 40)
    
    # Test 3 key steps
    key_steps = ["reason_for_call", "negotiation", "promise_to_pay"]
    
    user_id = "83906"
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    for step in key_steps:
        print(f"\n🎯 {step.upper()}")
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step=step,
            state=state.to_dict(),
            script_type="ratio_1_inflow",
            agent_name="Sarah"
        )
        
        prompt = get_step_prompt(step, parameters)
        
        # Show first 200 chars of prompt
        print(f"📝 PROMPT: {prompt[:200]}...")
        
        # Show key fields
        outstanding = parameters.get("outstanding_amount", "N/A")
        urgency = parameters.get("urgency_level", "N/A")
        print(f"💰 Amount: {outstanding} | ⚡ Urgency: {urgency}")

if __name__ == "__main__":
    print("🎯 CALL CENTER SYSTEM - COMPACT TESTS")
    print("=" * 60)
    
    # Choose which test to run
    test_choice = input("\nChoose test:\n1. All step prompts\n2. Script variations\n3. Conversation intelligence\n4. Outstanding calculation\n5. Quick test\n\nEnter number (1-5): ")
    
    if test_choice == "1":
        test_all_step_prompts()
    elif test_choice == "2":
        test_script_variations_short()
    elif test_choice == "3":
        test_conversation_intelligence()
    elif test_choice == "4":
        test_outstanding_calculation()
    elif test_choice == "5":
        quick_test()
    else:
        print("Running quick test by default...")
        quick_test()
    
    print("\n✅ Test completed!")

🎯 CALL CENTER SYSTEM - COMPACT TESTS


2025-05-29 14:00:00,405 - INFO - Using cached data for user_id: 83906
2025-05-29 14:00:00,407 - INFO - Verification field matched: id_number
2025-05-29 14:00:00,408 - INFO - Objection added: no_money
2025-05-29 14:00:00,409 - INFO - Emotional state updated to: cooperative
2025-05-29 14:00:00,411 - INFO - ================================================================================
2025-05-29 14:00:00,412 - INFO - PARAMETERS BUILT FOR USER: 83906, STEP: introduction
2025-05-29 14:00:00,413 - INFO - ================================================================================
2025-05-29 14:00:00,414 - INFO - Outstanding Amount: R 4030.00
2025-05-29 14:00:00,415 - INFO - Behavioral Analysis: friendly_reminder
2025-05-29 14:00:00,416 - INFO - Risk Level: high
2025-05-29 14:00:00,416 - INFO - Payment Reliability: high
2025-05-29 14:00:00,417 - INFO - ================================================================================
2025-05-29 14:00:00,418 - ERROR - Error formatting prom

🔄 Loading client data...
✅ Loaded data for: Linda Oelofse

🧪 Testing 16 call center steps:

📋 STEP  1: INTRODUCTION
--------------------------------------------------
🎯 PROMPT: 
[MISSING_BASE_CONTEXT]

<task>
Create professional first impression and establish contact with specific client.
</task>

<script_foundation>
Use this core language: "Good day, you are speaking to Sarah Mitchell from Cartrack Accounts Department. May I speak to Mrs Linda Oelofse, please?"
</script_foundation>

📊 KEY PARAMS: {'client_name': 'Linda', 'outstanding_amount': 'R 4030.00', 'urgency_level': 'high', 'key_motivators': 'Maintain vehicle security, keep account current'}
--------------------------------------------------

📋 STEP  2: NAME_VERIFICATION
--------------------------------------------------
🎯 PROMPT: 
[MISSING_BASE_CONTEXT]

<verification_status>
- Attempt: 0/5
- Current Status: VERIFIED
</verification_status>

<task>
Confirm client identity through name verification while building rapport.
</task>

In [6]:
user_id = "83906"
agent_name = "Sarah Mitchell"
script_type = "ratio_1_inflow"  # Standard first missed payment
client_data = get_client_data(user_id)


2025-05-27 13:11:42,719 - INFO - Using cached data for user_id: 83906


In [12]:
state = ConversationState()
state.add_objection("no_money")
state.update_emotional_state("defensive")
state.update_payment_willingness("low")
parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
reason_prompt = get_step_prompt("reason_for_call", parameters)
print(reason_prompt)

2025-05-27 13:13:03,915 - INFO - Objection added: no_money
2025-05-27 13:13:03,915 - INFO - Emotional state updated to: defensive
2025-05-27 13:13:03,916 - INFO - Payment willingness updated to: low
2025-05-27 13:13:03,916 - INFO - Built parameters for user_id: 83906, step: negotiation



<role>
You are Sarah Mitchell, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<context>
- Making OUTBOUND call regarding overdue account
- CRITICAL: Cannot discuss ANY account details until client identity is FULLY verified
- Client may be defensive, evasive, or emotional - remain professional but assertive
- Current step: negotiation
</context>

<communication_style>
- Professional, confident, and solution-focused
- Concise responses (max 30 words unless detailed explanation needed)
- Acknowledge emotions but redirect to resolution
- Use "I understand" to validate, then guide to next step
</communication_style>

<restrictions>
- NO account details or amounts until verification complete
- NO discussing reason for call until identity confirmed
- Stay focused but adapt tone to client response
</restrictions>

<task>
Clearly communicate account status and required payment amount.
</task>

<script_foundation>
Core message: ""
</script_foundation>

<c